# Week 7: Functions and Dynamic Scoping

We're introducing functions to Lettuce, which means we'll need a few new things:
- A way for functions to be defined
- A way for functions to be called

In order for us to have those as inference rules, we'll need to be able to save functions to our environment; that means updating our the Value rule in our grammar:

$$\begin{array}{rcl}
 \mathbf{Value} & \Rightarrow & Num(\mathbf{Double}) \\
 & \Rightarrow & Function({\mathbf{String^*}}, \mathbf{Expr}) \\
 & \Rightarrow & Error \\
\end{array}$$

Here, the String* within our Function value denotes parameter names.


Now we need to consider our inference rules for creating a function:

$$\newcommand\semRule[3]{\begin{array}{c} #1 \\ \hline #2 \\\end{array} (\text{#3})} $$
$$\newcommand\eval{\mathbf{eval}}$$
$$\semRule{}{\eval(\texttt{FunDef([id1,..., idk], e)},\sigma) = \text{Function}(\texttt{[id1,..., idk]}, \texttt{e})}{fundef}$$
<br /><br />

$$\semRule {\eval(\texttt{e}, \sigma) = \text{Function}(\texttt{[id1,..., idn]}, \texttt{fBody}), {n = k},\ (\forall\ i \in \{ 1, \ldots, k\})\ \eval(\texttt{ei}, \sigma) = v_i,  v_i \not= \mathbf{error}}
           {\eval(\texttt{FunCall(e, [e1, ..., ek])}, \sigma) = \eval(\texttt{fBody},{\sigma} {\circ [id1 \rightarrow v_1, \ldots, idk \rightarrow v_k]})}{funcall-ok}$$
     
So we have one rule that describes the ability to define a function.  Then we have our rule to call a function, which does the following:
- Evaluates all of the expressions passed in as parameters
- Updates the environment to ensure that parameter names point to the appropriate values
- Evaluates the function body on that new environment

We can also write some error rules:

$$\semRule {\eval(\texttt{e}, \sigma) \not\in \text{Function}}
           {\eval(\texttt{FunCall(e, [e1, ..., ek])}, \sigma) = \mathbf{error}}{funcall-not-a-function}$$
<br /><br />


$$\semRule {\eval(\texttt{e}, \sigma) = \text{Function}(\texttt{[id1,..., idn]}, \texttt{fBody}), {n \not= k}}
           {\eval(\texttt{FunCall(e, [e1, ..., ek])}, \sigma) = \mathbf{error}}{funcall-wrong-num-args}$$
           
  <br /><br />

$$\semRule {\eval(\texttt{e}, \sigma) = \text{Closure}(\texttt{[id1,..., idn]}, \texttt{fBody}), {n = k},\ (\exists\ i \in \{1, \ldots, k\})\ \eval(\texttt{ei}, \sigma) = \mathbf{error}}
           {\eval(\texttt{FunCall(e, [e1, ..., ek])}, \sigma) = \mathbf{error}}{funcall-arg-error}$$

# Exercise 1
Implement the necessary classes for our function inference rules

In [3]:
sealed trait Value
sealed trait Expr

case class NumValue(d: Double) extends Value
case class BoolValue(b: Boolean) extends Value
case object Error extends Value

//TODO: implement Function Value
case class Function(params: List[String], body: Expr) extends Value

case class Const(v: Double) extends Expr // Expr -> Const(v)
case class Ident(s: String) extends Expr // Expr -> Ident(s)

// Arithmetic Expressions
case class Plus(e1: Expr, e2: Expr) extends Expr // Expr -> Plus(Expr, Expr)

// Boolean Expressions
case class Geq(e1: Expr, e2:Expr) extends Expr

//If then else
case class IfThenElse(e: Expr, eIf: Expr, eElse: Expr) extends Expr

//Let bindings
case class Let(s: String, defExpr: Expr, bodyExpr: Expr) extends Expr

// BEGIN SOLUTION
case class FunCall(exp: Expr, params: List[String]) extends Expr
case class FunDef(params: List[String], fbody: Expr) extends Expr
// END SOLUTION

type Env = Map[String, Value]

defined trait Value
defined trait Expr
defined class NumValue
defined class BoolValue
defined object Error
defined class Function
defined class Const
defined class Ident
defined class Plus
defined class Geq
defined class IfThenElse
defined class Let
defined class FunCall
defined class FunDef
defined type Env

# Exercise 2
Implement the necessary cases for function definitions and calls within our evaluation logic.

In [ ]:
// Convenience functions for allowing modular implementation
type Eval = (Expr, Env) => Value

def getEvalExpr(evalLet: (Expr, Env, Eval) => Value, evalIdent: (Expr, Env, Eval) => Value): Eval = {
    def evalExpr(e: Expr, env: Env) : Value = e match {
        case Const(f) => NumValue(f)

        case Ident(x) => evalIdent(e, env, evalExpr)

        case Plus(e1, e2) => (evalExpr(e1, env), evalExpr(e2, env)) match {
                case (NumValue(n1), NumValue(n2)) => NumValue(n1 + n2)
                case _ => throw new IllegalArgumentException("Plus on non-number")
        }

        case Geq(e1, e2) => (evalExpr(e1, env), evalExpr(e2, env)) match {
            case (NumValue(n1), NumValue(n2)) => BoolValue( n1 >= n2)
            case _ => throw new IllegalArgumentException("Geq on non-number")
        }

        case IfThenElse(e1, e2, e3) => {
            val v = evalExpr(e1, env)
            v match {
                case BoolValue(true) => evalExpr(e2, env)
                case BoolValue(false) => evalExpr(e3, env)
                case _ => throw new IllegalArgumentException(s"If-then-else condition expr: ${e1} is non-boolean -- evaluates to ${v}")
            }
        }
        case e @ Let(_, _, _) => evalLet(e, env, evalExpr)
        
        // Hint: it might be worth thinking about map/zip/foldLeft with FunCall
        // BEGIN SOLUTION

        case FunDef(ids, e) =>  Function(ids, e)
        
        case FunCall(e, inputEx) => {
            val v = evalExpr(e, env) // this will evaluate the expression, put in param?
            v match {
                case Function(params, fBody) {
                    if (params.length == inputEx.length) {
                        val inVal = 
                    }

                }
            }
        }
        // END SOLUTION
        
        case _ => throw new IllegalArgumentException("Not supported")
    }
    
    evalExpr
}

def evalIdent1(identExpr : Expr, env: Env, evalExpr: (Expr, Env) => Value): Value = identExpr match {
    case Ident(x) => if (env contains x) env(x) else throw new IllegalArgumentException("Ill-formed")
    case _ => throw new IllegalArgumentException("Not a Let Expression")
}

def evalLet1(letExpr: Expr, env: Env, evalExpr: (Expr, Env) => Value): Value = letExpr match {
    case Let(x, e1, e2) => {
        val v1 = evalExpr(e1, env)
        val env2 = env + (x -> v1)
        evalExpr(e2, env2)
    }
    case _ => throw new IllegalArgumentException("Not a Let Expression")
}

(console):38:46 expected "=>"
                case Function(params, fBody) {
                                             ^

# Exercise 3
Let's test our logic with the programs below.

In [3]:
// Programs without function definitions/calls . . . we want to make sure we're not breaking
// some pre-existing logic
val p1 = Let("x", Const(3.0),
             Let("y", Const(2.0),
                 IfThenElse(Geq(Ident("x"), Ident("y")), Ident("x"), Ident("y"))
                 )
             )

val p2 = Let("x", Const(3.0),
             Let("x", Plus(Ident("x"), Const(1.0)),
               Ident("x")
            )
         )

val p3 = Let("x", Plus(Ident("x"), Const(3.0)),
                 Let("x", Plus(Ident("x"), Const(1.0)),
                     Ident("x")
                 )
         )
val p4 = Let("f", FunDef(List("var"), Plus(Ident("var"), Const(3.0))),
                 FunCall(Ident("f"), List(Const(2.0))))

cmd3.sc:21: type mismatch;
 found   : cmd3.this.cmd1.Const
 required: String
                 FunCall(Ident("f"), List(Const(2.0))))
                                               ^
Compilation Failed

Our programs above are as follows:

### Program 1
```ocaml
let x = 3 in 
    let y = 2 in 
        if x >= y 
            x
        else 
            y
```
Expected Value: 3

### Program 2
```ocaml
let x = 3 in 
    let x = x + 1 in
        x
```

Expected Value: 4
### Program 3
```ocaml
let x = x + 3 in 
    let x = x + 1 in
        x
```

### Program 4
```ocaml
let f = func(var) var + 3 in 
    f(2)
```

Expected Value: 5

In [ ]:
val evalExpr: Eval = getEvalExpr(evalLet1, evalIdent1)
assert(evalExpr(p1, Map.empty) == NumValue(3.0))
assert(evalExpr(p2, Map.empty) == NumValue(4.0))
try {
    evalExpr(p3, Map.empty)
    assert(false)
} catch {
    case e : IllegalArgumentException => if (e.getMessage == "Ill-formed") assert(true) else assert(false)
}

assert(evalExpr(p4, Map.empty) == NumValue(5.0))
println("It Worked!")

Now let's add a program to test multiple parameters:
### Program 5
```ocaml
let f = func(x, y) x + y in 
    f(2, 5)
```

Expected Value: 7

In [ ]:
val p5 = Let("f", FunDef(List("x", "y"), Plus(Ident("x"), Ident("y"))),
                 FunCall(Ident("f"), List(Const(2.0), Const(5.0))))

In [ ]:
assert(evalExpr(p5, Map.empty) == NumValue(7.0))

Now let's try referencing a global variable

### Program 6
```ocaml
let y = 2 in
    let f = func(x) x + y in 
        let y = 3 in
            f(2)
```

Expected Value: ???

What do we think this evaluates to?  Why?  Write a test case for this program; it should follow the assertion syntax above.

In [ ]:
val p6 = Let("y", Const(2.0), 
             Let("f", FunDef(List("x"), Plus(Ident("x"), Ident("y"))),
                Let("y", Const(3.0),
                    FunCall(Ident("f"), List(Const(2.0))
                    )
                )
            )
        )

// BEGIN SOLUTION
???
// END SOLUTION

# Dynamic Scope

What we've stumbled upon is what's known as dynamic scope.  This means that that, unlike most programming languages we're familiar with, when a function is called, it runs on the environment at runtime INSTEAD of running on the environment that existed when the function was defined.

As we can see, this might result in behavior that we don't expect, so it isn't necessarily desirable.  Instead, we'll be looking to implement static scope, which is what most of us are familiar with.  But in order to do so, we'll need to shake up our inference rules and create a few clever work arounds . . . and that is a task for the future.